# Import and define functions

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
import os

# https://stackoverflow.com/questions/75873241/how-does-camera-distortion-coefficients-and-camera-intrinsic-parameters-change-a
# distortion doesnt change

In [ ]:


def extract_intrinsics(file_path):
    # Parse the XML file and get the root element
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Navigate to the 'params' element
    params_element = root.find('.//camera_model/params')
    # Extract the value of the 'params' element
    params_text = params_element.text.strip(' [ ] ')

    # Split the value by semicolon and convert to float
    params = [float(param) for param in params_text.split(';')]

    return params

def extract_intrinsics_and_average(folder_path):
    # Get a list of all XML files in all subdirectories
    xml_files = glob.glob(folder_path, recursive=True)

    # Create an empty list to store all parameters
    all_params = []

    # Iterate over the list of XML files
    for xml_file in xml_files:
        # Extract the intrinsics and append them to all_params
        params = extract_intrinsics(xml_file)
        all_params.append(params)

    # Convert all_params to a NumPy array
    all_params_array = np.array(all_params)

    # Calculate the average of the array along the first axis
    average_params = np.mean(all_params_array, axis=0)

    return average_params, all_params_array


def adjust_intrinsics_for_crop(intrinsics, crop_left, crop_top):
    # Adjust the principal point
    intrinsics[2] -= crop_left  # cx
    intrinsics[3] -= crop_top  # cy

    return intrinsics

def adjust_intrinsics_for_new_resolution(intrinsics, old_size, new_size):
    # Calculate the scale factors
    width_scale = new_size[0] / old_size[0]
    height_scale = new_size[1] / old_size[1]

    # Adjust the intrinsics
    intrinsics[0] *= width_scale  # fx
    intrinsics[1] *= height_scale  # fy
    intrinsics[2] *= width_scale  # cx
    intrinsics[3] *= height_scale  # cy

    return intrinsics

def normalize_intrinsics(intrinsics, image_size):
    # Unpack the intrinsics
    fx, fy, cx, cy = intrinsics

    # Normalize the intrinsics
    fx /= image_size[0]  # image_width
    fy /= image_size[1]  # image_height
    cx /= image_size[0]  # image_width
    cy /= image_size[1]  # image_height

    return [fx, fy, cx, cy]


# C3VD

## C3VD to HK intrinsics conversion

In [ ]:

# HK 720x576 -> 576x576 -> 288x288
# to do C3VD to HK 1350x1080 -> 720x576 then crop to 576x576 then resize to 288x288


# Original size and intrinsics
C3VD_size = (1350, 1080)
intrinsics = [802.319, 801.885, 668.286, 547.733,]# dimi: [767.3861511125845, 767.5058656118406, 679.054265997005, 543.646891684636]#
distortions = [-0.42234, 0.10654, 0, 0] # dimi: [-0.18867185058223412, -0.003927337093919806, 0.030524814153620117, -0.012756926010904904]#

# Resize to 720x576
HK_size = (720, 576)
intrinsics = adjust_intrinsics_for_new_resolution(intrinsics, C3VD_size, HK_size)

# Crop to 576x576
cropped_size = (576, 576) #1080/1.875
crop_left = (720 - cropped_size[0]) / 2
crop_top = (576 - cropped_size[1]) / 2
intrinsics = adjust_intrinsics_for_crop(intrinsics, crop_left, crop_top)

# Resize to 288x288
HK_modified_size = (288, 288)
intrinsics = adjust_intrinsics_for_new_resolution(intrinsics, cropped_size, HK_modified_size)

print("intrinsics", intrinsics) # [188.42553611 201.35722074 150.06753194 143.08104148]

# Normalize the intrinsics
normalized_intrinsics = normalize_intrinsics(intrinsics, HK_modified_size)

print("normalized_intrinsics",normalized_intrinsics) #[0.6542553337152778, 0.6991570164583334, 0.5210678192361111, 0.4968091718055556]
print("distortions",distortions)

In [ ]:
# https://github.com/DurrLab/C3VD/issues/15

FOV = -1.1 # -1 is too zoomed in 
camera_matrix = np.array([[intrinsics[0], 0, intrinsics[2]],
                           [0, intrinsics[1], intrinsics[3]],
                           [0, 0, 1]])
dist_coeffs = np.array([distortions])
# Load the image
image = cv2.imread('/media/rema/data/DataHKGab/Normal/1/00000.jpg')

# # Undistort the image
K_new = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(camera_matrix, dist_coeffs, (288,288), np.eye(3), balance=1)
map1, map2 = cv2.fisheye.initUndistortRectifyMap(camera_matrix, dist_coeffs, np.eye(3), K_new, (288,288), cv2.CV_32FC1)

# Undistort all pixels
K_new, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (288,288), FOV, (288, 288))

print("Old intrinsics", intrinsics)
print("New intrinsics", K_new[0,0], K_new[1,1], K_new[0,2], K_new[1,2])
print("New intrinsics normalized", normalize_intrinsics([K_new[0,0], K_new[1,1], K_new[0,2], K_new[1,2]], (288,288)))
print("Distortions", dist_coeffs)


# Finally we used C3VD intrinsics and distortions:

old_intrinsics = [213.95173333333332, 213.83599999999998, 142.2096, 146.06213333333332]  
new_intrinsics = [184.98585502685236, 184.35666485498837, 138.93697343164294, 152.59259458599243]  
New intrinsics normalized [0.6423119966210151, 0.6401273085242651, 0.4824200466376491, 0.5298353978680292]  
distortions = [-0.42234, 0.10654, 0, 0]

### Undistort images

In [ ]:
def undistort_files(image_files, K_old, K_new, dist_coeffs, foldername):
    for image_file in image_files:
        # Create the output directory if it doesn't exist
        output_dir = os.path.dirname(image_file).replace(f'{foldername}', f'{foldername}/Undistorted')
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, os.path.basename(image_file).replace('.jpg', '.png'))
        if os.path.exists(output_file):
            continue
        # Read the image
        img = cv2.imread(image_file)
        if foldername == "C3VD":
            image_resize = cv2.resize(img, (720,576))

            height, width, _ = image_resize.shape
            sides = height if height < width else width
            hrem=height-sides
            wrem=width-sides

            # Resizing the images
            imCrop=image_resize[int(hrem/2):int(height-hrem/2),int(wrem/2):int(width-wrem/2),:]
            img=cv2.resize(imCrop, (288,288))

        # shuai said use this because his calibration is pinhole
        undistorted_img = cv2.undistort(img, K_old, dist_coeffs, None, K_new)
        
        # Save the undistorted image
        print(output_file)
        cv2.imwrite(output_file, undistorted_img)
        
def undistort_files_aux(image_file, img, K_old, K_new, dist_coeffs, foldername, binarize=False):
        # shuai said use this because his calibration is pinhole
        undistorted_img = cv2.undistort(img, K_old, dist_coeffs, None, K_new)
        
        if binarize:
            # binarize image to 0 and 255 with threshold 200
            _, undistorted_img = cv2.threshold(undistorted_img, 200, 255, cv2.THRESH_BINARY)
            
        # Create the output directory if it doesn't exist
        output_dir = os.path.dirname(image_file).replace(f'{foldername}', f'{foldername}/Undistorted')
        os.makedirs(output_dir, exist_ok=True)

        # Save the undistorted image
        output_file = os.path.join(output_dir, os.path.basename(image_file).replace('.jpg', '.png'))
        print(output_file)
        cv2.imwrite(output_file, undistorted_img)

In [ ]:
# Define the distortion coefficients and camera matrix
# These values are usually obtained from camera calibration
old_intrinsics = [213.95173333333332, 213.83599999999998, 142.2096, 146.06213333333332]
new_intrinsics = [184.98585502685236, 184.35666485498837, 138.93697343164294, 152.59259458599243]
distortions = [-0.42234, 0.10654, 0, 0]

K_old = np.array([[old_intrinsics[0], 0, old_intrinsics[2]],
                  [0, old_intrinsics[1], old_intrinsics[3]],
                  [0, 0, 1]])
K_new = np.array([[new_intrinsics[0], 0, new_intrinsics[2]],
                  [0, new_intrinsics[1], new_intrinsics[3]],
                  [0, 0, 1]])
dist_coeffs = np.array([distortions])



#### C3VD

In [ ]:
# Get a list of all the image file paths
image_files = glob.glob('../Datasets/C3VD/Dataset/*/*color.png')
foldername = "C3VD"
undistort_files(image_files, K_old, K_new, dist_coeffs, foldername)

In [ ]:
#Depth 
image_files = glob.glob('../Datasets/C3VD/Dataset/*/*depth.tiff')
foldername = "C3VD"

for image_file in image_files:
    # Create the output directory if it doesn't exist
    output_dir = os.path.dirname(image_file).replace(f'{foldername}', f'{foldername}/Undistorted')
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, os.path.basename(image_file).replace('.jpg', '.png'))
    if os.path.exists(output_file):
        continue
    # Read the image
    img = cv2.imread(image_file, cv2.IMREAD_UNCHANGED)
    
    
    image_resize = cv2.resize(img, (720,576))

    height, width = image_resize.shape
    sides = height if height < width else width
    hrem=height-sides
    wrem=width-sides

    # Resizing the images
    imCrop=image_resize[int(hrem/2):int(height-hrem/2),int(wrem/2):int(width-wrem/2)]
    img_final=cv2.resize(imCrop, (288,288))
    
    #undistort
    undistort_files_aux(image_file, img_final, K_old, K_new, dist_coeffs, foldername)

#### BBPS

In [ ]:
# Get a list of all the image file paths
image_files = glob.glob('../Datasets/BBPS-2-3Frames/Frames/*/*.jpg')
foldername = "BBPS-2-3Frames"
undistort_files(image_files, K_old, K_new, dist_coeffs, foldername)